In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import geopandas as gpd
from shapely.geometry import Point
import datetime
import requests
import pandas as pd
import os
import sys
from osgeo import gdal
import glob
import pickle
import xarray as xr
import pyarrow

# read shape files

In [ ]:

# soa_shape_map_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/doc_2024-03-26_15-21-36.gridpoint_2000.shp/.gridpoint_2000.shp.shp'
# sub_basin_path = '/content/drive/MyDrive/import/Central_valley/central_valley_3857.shp'
sub_basin_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/central_valley_3857/central_valley_3857.shp'
# mesh_grid_path = "/content/drive/MyDrive/import/prism_mesh/prism_4km_mesh.shp"
mesh_grid_path = "/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/prism_4km_mesh/prism_4km_mesh.shp"
# soa_shape_map = gpd.read_file(soa_shape_map_path)
sub_basin_map = gpd.read_file(sub_basin_path)

mesh = gpd.read_file(mesh_grid_path)

# soa_shape_map_geo = soa_shape_map.to_crs(epsg=4326)
sub_basin_map_geo = sub_basin_map.to_crs(epsg=4326)
# mesh_geo = mesh.to_crs(epsg=4326)

# sub_basin_1 = sub_basin_map_geo[sub_basin_map_geo['Subregion_'] == 1]

mesh


ERROR:fiona._env:/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/central_valley_3857/central_valley_3857.shp: No such file or directory


DriverError: /home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/central_valley_3857/central_valley_3857.shp: No such file or directory

## sample

In [ ]:
clipped_mesh = gpd.clip(mesh, sub_basin_map_geo)

# mesh_center = mesh[(mesh['col'] == 71) & (mesh['row'] == 220)]
# mesh_up = mesh[(mesh['col'] == 71) & (mesh['row'] == 219)]
# mesh_down = mesh[(mesh['col'] == 71) & (mesh['row'] == 221)]
# mesh_right = mesh[(mesh['col'] == 72) & (mesh['row'] == 220)]
# mesh_left = mesh[(mesh['col'] == 70) & (mesh['row'] == 220)]

clipped_mesh

/tmp/ipykernel_11777/1449953008.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: EPSG:4326

  clipped_mesh = gpd.clip(mesh, sub_basin_map_geo)


,col,row,lon,lat,geometry
367931,145,360,-118.958333,34.916667,"POLYGON ((-118.93750 34.93750, -118.93750 34.9..."
367933,147,360,-118.875000,34.916667,"POLYGON ((-118.88363 34.93750, -118.89583 34.9..."
367930,144,360,-119.000000,34.916667,"POLYGON ((-118.97917 34.93750, -118.97917 34.9..."
367932,146,360,-118.916667,34.916667,"POLYGON ((-118.89583 34.93750, -118.89583 34.9..."
366865,149,359,-118.791667,34.958333,"POLYGON ((-118.79617 34.97917, -118.81250 34.9..."
...,...,...,...,...,...
211328,66,223,-122.250000,40.625000,"POLYGON ((-122.24930 40.64583, -122.22917 40.6..."
211329,67,223,-122.208333,40.625000,"POLYGON ((-122.22917 40.60417, -122.22917 40.6..."
211327,65,223,-122.291667,40.625000,"POLYGON ((-122.27083 40.64583, -122.27083 40.6..."
210092,65,222,-122.291667,40.666667,"POLYGON ((-122.27083 40.64583, -122.29115 40.6..."


## create a point layer and list of coordinates

In [ ]:
# copy poly to new GeoDataFrame
points = clipped_mesh.copy()
# change the geometry
points.geometry = points['geometry'].centroid
# same crs
points.crs =clipped_mesh.crs
points.head()

/tmp/ipykernel_11777/1104426375.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points.geometry = points['geometry'].centroid


,col,row,lon,lat,geometry
367931,145,360,-118.958333,34.916667,POINT (-118.95388 34.92906)
367933,147,360,-118.875000,34.916667,POINT (-118.89177 34.93587)
367930,144,360,-119.000000,34.916667,POINT (-118.98272 34.93581)
367932,146,360,-118.916667,34.916667,POINT (-118.92105 34.92998)
366865,149,359,-118.791667,34.958333,POINT (-118.80706 34.97626)


In [ ]:


# Create a list of points with latitude and longitude
# points = [
#     Point(lon, lat) for lon, lat in [
#         (mesh_center["lon"].item(), mesh_center["lat"].item()),
#         (mesh_up["lon"].item(), mesh_up["lat"].item()),
#         (mesh_down["lon"].item(), mesh_down["lat"].item()),
#         (mesh_right["lon"].item(), mesh_right["lat"].item()),
#         (mesh_left["lon"].item(), mesh_left["lat"].item())
#         # Add more points as needed
#     ]
# ]

# Create a GeoDataFrame from the list of points
# shape_four = gpd.GeoDataFrame(geometry=points)

lat_lon_tuples_california = list(zip(points.geometry.x, points.geometry.y))
len(lat_lon_tuples_california)

3314

## save this list as a shape file

In [ ]:
# clipped_points = gpd.clip(soa_shape_map_geo, sub_basin_1)

# lat_lon_tuples = list(zip(clipped_points.geometry.y, clipped_points.geometry.x))

# Create a GeoDataFrame from the list of points
geometry = [Point(lon, lat) for lon, lat in lat_lon_tuples_california]
gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")


# Save the GeoDataFrame as a shapefile
gdf.to_file('/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/lat_lon_complete/shape_complete.shp')


# generate time series

In [ ]:


def generate_time_series(start_date, end_date, freq):
    """
    Generate a time series of dates in the format "YYYYMMDD".

    Args:
        start_date (str): The start date in the format "YYYY-MM-DD".
        end_date (str): The end date in the format "YYYY-MM-DD".
        freq (str): The frequency of the time series. Possible values are
            "D" (daily), "W" (weekly), "M" (monthly), etc.

    Returns:
        list: A list of dates in the format "YYYYMMDD".
    """
    date_series = []
    start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    current_date = start

    while current_date <= end:
        date_series.append(current_date.strftime("%Y%m%d"))
        current_date += datetime.timedelta(days=1) if freq == "D" else datetime.timedelta(weeks=1) if freq == "W" else datetime.timedelta(days=30.4375)

    return date_series

# function of download prism data

In [ ]:


def download_file(url):
  """Downloads a file from the given URL and saves it locally."""
  local_filename = f"/content/drive/MyDrive/export/prism/{url.split('/')[-1]}"
  # NOTE the stream=True parameter below
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
      for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)
  return local_filename


def download_prism_data(lat_lon_tuples, output_folder):
  """Downloads PIRISM data for a list of latitude and longitude tuples.

  Args:
    lat_lon_tuples: A list of tuples containing (latitude, longitude) coordinates.
    output_folder: The folder where downloaded data files will be saved.
  """
  session = requests.Session()
  base_url = "https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php"
  data = {
    "spares": "4km",
    "interp": "0",
    "stats": "ppt tmin tmean tmax",  # You can customize the variables here
    "units": "si",
    "range": "daily",
    "start": "19910101",
    "end": "20201231",
    "stability": "stable",
    "call": "pp/daily_timeseries",
    "proc": "gridserv"
  }
  headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0",
    "DNT": "1",
    "Sec-GPC": "1",
  }

  for lon, lat in lat_lon_tuples:
    data["lat"] = str(lat)
    data["lon"] = str(lon)
    response = session.post(base_url, data=data, headers=headers)
    res_json = response.json()


    download_url = "https://prism.oregonstate.edu/explorer/tmp/" + res_json["result"]["csv"]
    filename = f"{output_folder}/{lon}_{lat}.csv"  # Customize filename format
    download_file(download_url)
    print(f"Downloaded data for ({lon}, {lat}) to {filename}")



## example

In [ ]:
# Example usage
output_folder = "/content/drive/MyDrive/export/prism"  # Change this if needed
download_prism_data(lat_lon_tuples_california, output_folder)

Downloaded data for (34.92906226268493, -118.9538819077391) to /content/drive/MyDrive/export/prism/34.92906226268493_-118.9538819077391.csv
Downloaded data for (34.93586509830011, -118.89176660527282) to /content/drive/MyDrive/export/prism/34.93586509830011_-118.89176660527282.csv
Downloaded data for (34.93581177674317, -118.98271712602246) to /content/drive/MyDrive/export/prism/34.93581177674317_-118.98271712602246.csv
Downloaded data for (34.929979594899265, -118.92104633728002) to /content/drive/MyDrive/export/prism/34.929979594899265_-118.92104633728002.csv
Downloaded data for (34.976262831426034, -118.80705582222832) to /content/drive/MyDrive/export/prism/34.976262831426034_-118.80705582222832.csv
Downloaded data for (35.001786012110195, -118.79302168805502) to /content/drive/MyDrive/export/prism/35.001786012110195_-118.79302168805502.csv
Downloaded data for (35.01103635213238, -119.32921437097426) to /content/drive/MyDrive/export/prism/35.01103635213238_-119.32921437097426.csv
Do

# use pyPRISMClimate library and FTP

In [ ]:

sys.path.append('/home/mahdi/data/pyPRISMClimate')
from pyPRISMClimate.quick_tools import *

## download daily data

In [ ]:
data = get_prism_dailys(
    variable ="tmean",
    min_date="1991-01-01",
    max_date="2020-12-31",
    dest_path="/home/mahdi/data/export"
)

### open bil file

In [ ]:


# Open the BIL file
dataset = gdal.Open("/home/mahdi/data/export/PRISM_tmean_stable_4kmD2_20201227_bil.bil")

### convert to array

In [ ]:
# Get the number of bands and data type
num_bands = dataset.RasterCount
data_type = dataset.GetRasterBand(1).DataType
# Read the data as a NumPy array
data = dataset.GetRasterBand(1).ReadAsArray()
# (Optional) Get geotransform information
geotransform = dataset.GetGeoTransform()
geotransform

# read shorted lat and lon for shape file

In [ ]:
arr = [f for f in os.listdir('/content/drive/MyDrive/w/Prism') if f.endswith(".csv")]
arr

['PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2083_-121.1250.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.1719_-120.8500.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2110_-120.8354.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2214_-120.8008.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2083_-120.8750.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2083_-121.0417.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2083_-120.9583.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2083_-121.0833.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2500_-121.1250.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2500_-121.0000.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2500_-121.0833.csv',
 'PRISM_ppt_tmin_tmean_tmax_stable_4km_19910101_20201231_38.2500_-120.9167.csv',
 'PRISM_ppt_tmin_tmean_tmax_

## list of tuples lon and lat

In [ ]:
points_list = []


for i, s in enumerate(arr):
    print(f"i: {i}, Lon: {s[-13:-4]}, Lat: {s[-21:-14]}")
    lat = float(s[-21:-14])
    lon = float(s[-13:-4])

    points_list.append(tuple((lon, lat)))

i: 0, Lon: -121.1250, Lat: 38.2083
i: 1, Lon: -120.8500, Lat: 38.1719
i: 2, Lon: -120.8354, Lat: 38.2110
i: 3, Lon: -120.8008, Lat: 38.2214
i: 4, Lon: -120.8750, Lat: 38.2083
i: 5, Lon: -121.0417, Lat: 38.2083
i: 6, Lon: -120.9583, Lat: 38.2083
i: 7, Lon: -121.0833, Lat: 38.2083
i: 8, Lon: -121.1250, Lat: 38.2500
i: 9, Lon: -121.0000, Lat: 38.2500
i: 10, Lon: -121.0833, Lat: 38.2500
i: 11, Lon: -120.9167, Lat: 38.2500
i: 12, Lon: -120.8750, Lat: 38.2500
i: 13, Lon: -120.9583, Lat: 38.2500
i: 14, Lon: -121.0417, Lat: 38.2500
i: 15, Lon: -120.8019, Lat: 38.2378
i: 16, Lon: -120.8347, Lat: 38.2485
i: 17, Lon: -120.8468, Lat: 38.2764
i: 18, Lon: -120.8920, Lat: 38.3156
i: 19, Lon: -121.0417, Lat: 38.2917
i: 20, Lon: -120.9167, Lat: 38.2917
i: 21, Lon: -120.8780, Lat: 38.2882
i: 22, Lon: -121.1250, Lat: 38.2917
i: 23, Lon: -120.9583, Lat: 38.2917
i: 24, Lon: -121.0000, Lat: 38.2917
i: 25, Lon: -121.0833, Lat: 38.2917
i: 26, Lon: -120.9583, Lat: 38.3333
i: 27, Lon: -121.1250, Lat: 38.3333
i:

## save this list as a shape file

In [ ]:




# Create a GeoDataFrame from the list of points
geometry = [Point(lon, lat) for lon, lat in points_list]
gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")

# Save the GeoDataFrame as a shapefile
gdf.to_file('/content/drive/MyDrive/w/Prism/import_shape/shape_shorted.shp')

# merge all of excel files in a specfic directory

In [3]:
# Specify the directory path and file pattern

file_path = '/content/drive/MyDrive/data/Prism/*.csv'
csv_files = glob.glob(file_path)

dfs = []

for file in csv_files:
    # Read the first 11 rows as attributes
    # attributes = pd.read_csv(file, nrows=11, header=None)

    # Read the time series data, skipping the first 11 rows
    ts_data = pd.read_csv(file, skiprows=10)

    lat_v = float(file[-21:-14])
    lon_v = float(file[-13:-4])
    ts_data = ts_data.assign(latitude = lat_v, longitude = lon_v)
    # Add the attributes as new columns to the time series data
    # for i in range(attributes.shape[1]):
    #     ts_data[f'Attribute_{i+1}'] = attributes.iloc[:, i].values

    dfs.append(ts_data)

merged_df = pd.concat(dfs, ignore_index=True)
merged_df

,Date,ppt (mm),tmin (degrees C),tmean (degrees C),tmax (degrees C),latitude,longitude
0,1991-01-01,0.00,-1.1,5.3,11.6,38.1719,-120.8500
1,1991-01-02,0.00,-0.7,5.8,12.2,38.1719,-120.8500
2,1991-01-03,0.00,-0.3,6.4,13.1,38.1719,-120.8500
3,1991-01-04,0.00,1.9,6.6,11.3,38.1719,-120.8500
4,1991-01-05,0.00,3.3,7.2,11.0,38.1719,-120.8500
...,...,...,...,...,...,...,...
36314807,2020-12-27,1.55,6.0,11.3,16.6,35.5417,-119.8333
36314808,2020-12-28,4.96,7.1,11.7,16.3,35.5417,-119.8333
36314809,2020-12-29,3.08,5.5,8.1,10.7,35.5417,-119.8333
36314810,2020-12-30,0.00,-0.1,6.5,13.1,35.5417,-119.8333


## save dataframe as a parquet file

In [4]:

merged_df.to_parquet("/content/drive/MyDrive/data/Prism/merged_files/merged_prism_data_par.parquet", engine="pyarrow")

In [6]:
tt = pd.read_parquet('/content/drive/MyDrive/data/Prism/merged_files/merged_prism_data_par.parquet', engine='pyarrow')
tt

,Date,ppt (mm),tmin (degrees C),tmean (degrees C),tmax (degrees C),latitude,longitude
0,1991-01-01,0.00,-1.1,5.3,11.6,38.1719,-120.8500
1,1991-01-02,0.00,-0.7,5.8,12.2,38.1719,-120.8500
2,1991-01-03,0.00,-0.3,6.4,13.1,38.1719,-120.8500
3,1991-01-04,0.00,1.9,6.6,11.3,38.1719,-120.8500
4,1991-01-05,0.00,3.3,7.2,11.0,38.1719,-120.8500
...,...,...,...,...,...,...,...
36314807,2020-12-27,1.55,6.0,11.3,16.6,35.5417,-119.8333
36314808,2020-12-28,4.96,7.1,11.7,16.3,35.5417,-119.8333
36314809,2020-12-29,3.08,5.5,8.1,10.7,35.5417,-119.8333
36314810,2020-12-30,0.00,-0.1,6.5,13.1,35.5417,-119.8333


# save as a xarray

In [ ]:
tt['time'] = pd.to_datetime(tt['Date'],format= "%Y-%m-%d" )
tt = tt.drop('Date', axis=1)
tt

,ppt (mm),tmin (degrees C),tmean (degrees C),tmax (degrees C),latitude,longitude,time
0,0.00,-4.0,3.7,11.4,38.2083,-121.125,1991-01-01
1,0.00,-3.5,4.1,11.7,38.2083,-121.125,1991-01-02
2,0.00,-2.5,4.6,11.6,38.2083,-121.125,1991-01-03
3,0.00,-0.2,5.3,10.9,38.2083,-121.125,1991-01-04
4,0.00,-0.8,4.9,10.5,38.2083,-121.125,1991-01-05
...,...,...,...,...,...,...,...
36314807,0.00,6.3,11.6,16.9,35.5417,-119.625,2020-12-27
36314808,4.13,7.3,12.0,16.6,35.5417,-119.625,2020-12-28
36314809,3.59,6.0,8.7,11.5,35.5417,-119.625,2020-12-29
36314810,0.00,0.1,6.7,13.4,35.5417,-119.625,2020-12-30


In [ ]:
list(tt.columns.values)

['ppt (mm)',
 'tmin (degrees C)',
 'tmean (degrees C)',
 'tmax (degrees C)',
 'latitude',
 'longitude',
 'time']

In [ ]:
tt.columns

Index(['ppt (mm)', 'tmin (degrees C)', 'tmean (degrees C)', 'tmax (degrees C)',
       'latitude', 'longitude', 'time'],
      dtype='object')

In [ ]:
tt.rename({'ppt (mm)': 'p',
           'tmin (degrees C)': 't_min',
           'tmean (degrees C)': 't_mean',
           'tmax (degrees C)': 't_max',
           'latitude': 'lat',
           'longitude': 'lon'}, axis=1, inplace=True)

In [ ]:

tt.index.names = ['time','lat','lon']
ds = tt.to_xarray()
ds

ValueError: Length of new names must be 1, got 3

In [ ]:
data_array = tt.to_xarray()

data_array.set_coords(['time','latitude','longitude'])

<xarray.Dataset>
Dimensions:            (index: 36314812)
Coordinates:
  * index              (index) int64 0 1 2 3 ... 36314809 36314810 36314811
    latitude           (index) float64 38.21 38.21 38.21 ... 35.54 35.54 35.54
    longitude          (index) float64 -121.1 -121.1 -121.1 ... -119.6 -119.6
    time               (index) datetime64[ns] 1991-01-01 ... 2020-12-31
Data variables:
    ppt (mm)           (index) float64 0.0 0.0 0.0 0.0 ... 4.13 3.59 0.0 1.81
    tmin (degrees C)   (index) float64 -4.0 -3.5 -2.5 -0.2 ... 7.3 6.0 0.1 -0.7
    tmean (degrees C)  (index) float64 3.7 4.1 4.6 5.3 4.9 ... 12.0 8.7 6.7 6.8
    tmax (degrees C)   (index) float64 11.4 11.7 11.6 10.9 ... 11.5 13.4 14.3

In [ ]:
da_nn = data_array.astype('datetime64',copy=False)
da_nn

<ipython-input-13-f711341f6691>:1: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_nn = data_array.astype('datetime64',copy=False)


TypeError: datetime64 values must have a unit specified